## Generating Trade Signals Using Technical Indicators with XGBoost

Trade signal generation involves identifying buy or sell signals based on historical price data and technical indicators.

By training an XGBoost model on past market data, we can predict future price movements and generate trade signals.

For more details, please read the corresponding Medium article [here](https://medium.com/@konstantinos.mountris/generating-trade-signals-using-technical-indicators-with-xgboost-9b78f18e3b4d).

The general workflow involves the following steps:
1. Data Collection
2. Feature Engineering
3. Model Training
4. Signal Generation

### Step 1: Data Collection

In [68]:
import yfinance as yf
import pandas as pd

# Fetch historical data for a stock (e.g., NVIDIA)
ticker = 'NVDA'
data = yf.download(
    ticker,
    start='2020-01-01',
    end='2023-01-01',
    interval='1d',
    progress=False)

# Display descriptive statistics for the stock
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,756.000000,756.000000,756.000000,756.000000,756.000000,7.560000e+02
mean,15.982999,16.306313,15.637073,15.982407,15.953369,4.611100e+08
std,6.373669,6.514503,6.189719,6.348385,6.343381,1.982675e+08
min,5.002500,5.248500,4.517000,4.910000,4.893182,9.788400e+07
25%,12.506500,12.733750,12.164125,12.515625,12.492046,2.990020e+08
50%,14.719000,14.937875,14.297750,14.613250,14.600136,4.410060e+08
75%,19.873126,20.129250,19.435750,19.856625,19.820983,5.804800e+08
max,33.516998,34.646999,32.035999,33.375999,33.321648,1.463684e+09


### Step 2: Feature Engineering

We use the following common indicators as features for our model:
* Simple Moving Average (SMA)
* Exponential Moving Average (EMA)
* Relative Strength Index (RSI)
* Moving Average Convergence Divergence (MACD)
* Bollinger Bands (BB_High/BB_Low)

In [57]:
import ta

# Calculate technical indicators
data['SMA'] = ta.trend.sma_indicator(data['Close'], window=20)
data['EMA'] = ta.trend.ema_indicator(data['Close'], window=20)
data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
data['MACD'] = ta.trend.macd(data['Close'])
data['MACD_Signal'] = ta.trend.macd_signal(data['Close'])
data['BB_High'] = ta.volatility.bollinger_hband(data['Close'], window=20)
data['BB_Low'] = ta.volatility.bollinger_lband(data['Close'], window=20)

# Drop rows with missing values
data.dropna(inplace=True)

### Step 3: Model Training

In [66]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np

# Define the target variable:
# 1 for price increase,
# 0 for price decrease
data['Target'] = np.where(data['Close'].shift(-1) > data['Open'], 1, 0)

# Select features and target
features = ['SMA', 'EMA', 'RSI', 'MACD', 'MACD_Signal', 'BB_High', 'BB_Low']
X = data[features]
y = data['Target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy:.2f}')

Model Accuracy: 0.68


### Step 4: Signal Generation

In [67]:
# Generate trade signals
pred = model.predict(X)
data['Signal'] = np.where(pred == 1, 'Buy', 'Sell')

# Display the trade signals for the last few days
print(data[['Open','Close','Signal']].tail())

                             Open   Close Signal
Date                                            
2022-12-23 00:00:00-05:00  15.196  15.206    Buy
2022-12-27 00:00:00-05:00  15.074  14.121   Sell
2022-12-28 00:00:00-05:00  13.927  14.036    Buy
2022-12-29 00:00:00-05:00  14.402  14.603   Sell
2022-12-30 00:00:00-05:00  14.334  14.614   Sell
